In [3]:
import torch
torch.cuda.is_available(), torch.__version__

(False, '2.8.0+cu128')

In [4]:
def square(x):
    return x * x

In [5]:
type(square)

function

In [6]:
square(3)

9

In [2]:
import tiktoken
tiktoken.__version__

'0.12.0'

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
tokenizer.encode("Hello, world!")


[15496, 11, 995, 0]

In [5]:
tokenizer.decode(tokenizer.encode("Hello, world!"))

'Hello, world!'

In [8]:
import torch
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cpu')

In [18]:
torch.__version__

'2.8.0+cu128'

In [19]:
from torch import nn
from torch.nn import functional as F